# Welcome to the Google Earth Engine (GEE) Python Introductory tutorial for Omdena Challenge.

### The purpose is to allow collaborators who have no or little information with regards to Python and GEE relevant for use in this challenge.


## Basic Approach:
1. Install required libraries.
2. Load libraries.
3. Load shapefile.  
   Create the shapefile from QGIS or something similar.  Need to do this if extracting certain geometry.
4. Inspect and extract the necessary features from the shapefile relevant to the analysis.
5. Convert the GeoDataFrame into GeoJSON format to use with GEE.
6. Select satellite attributes, i.e. MODIS? Landsat?
7. Display or export raster/s

In [ ]:
!pip -q install numpy
!pip -q install matplotlib
!pip -q install pandas
!pip -q install geopandas
!pip -q install rasterio
!pip -q install rasterstats
!pip -q install httpx
!pip -q install owslib
!pip -q install earthengine-api
!pip -q install jupyter_contrib_nbextensions
!pip -q install yapf
!pip -q install pandas-profiling
!pip -q install opencv-python
!pip -q install boto3
!pip -q install botocore
!pip -q install awscli

In [ ]:
# 2. Load libraries
# base libraries
import numpy as np
import pandas as pd 
from scipy import ndimage

# visualization libraries and setup
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
pylab.rcParams['figure.figsize'] = 8, 6
%matplotlib inline

# geospatial libraries
import geopandas as gpd
import ee

In [ ]:
#initialize google earth engine
import ee
try:
  ee.Initialize()
except:
  ee.Authenticate()
  ee.Initialize()

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
shape_file = './drive/My Drive/Netherlands/shapefiles/NLD_adm0_single.shp'

In [ ]:
# read the shapefile, ensure correct name
shapefile = gpd.GeoDataFrame.from_file(shape_file)

4. Data examination, these are suggested steps.

In [ ]:
shapefile.shape

(18, 71)

In [ ]:
shapefile.columns

Index(['ID_0', 'ISO', 'NAME_0', 'OBJECTID_1', 'ISO3', 'NAME_ENGLI', 'NAME_ISO',
       'NAME_FAO', 'NAME_LOCAL', 'NAME_OBSOL', 'NAME_VARIA', 'NAME_NONLA',
       'NAME_FRENC', 'NAME_SPANI', 'NAME_RUSSI', 'NAME_ARABI', 'NAME_CHINE',
       'WASPARTOF', 'CONTAINS', 'SOVEREIGN', 'ISO2', 'WWW', 'FIPS', 'ISON',
       'VALIDFR', 'VALIDTO', 'POP2000', 'SQKM', 'POPSQKM', 'UNREGION1',
       'UNREGION2', 'DEVELOPING', 'CIS', 'Transition', 'OECD', 'WBREGION',
       'WBINCOME', 'WBDEBT', 'WBOTHER', 'CEEAC', 'CEMAC', 'CEPLG', 'COMESA',
       'EAC', 'ECOWAS', 'IGAD', 'IOC', 'MRU', 'SACU', 'UEMOA', 'UMA', 'PALOP',
       'PARTA', 'CACM', 'EurAsEC', 'Agadir', 'SAARC', 'ASEAN', 'NAFTA', 'GCC',
       'CSN', 'CARICOM', 'EU', 'CAN', 'ACP', 'Landlocked', 'AOSIS', 'SIDS',
       'Islands', 'LDC', 'geometry'],
      dtype='object')

In [ ]:
shapefile['NAME_0'].unique()

array(['Netherlands'], dtype=object)

In [ ]:
shapefile.info()

In [ ]:
shapefile.head()

In [ ]:
shapefile.shape[0]

18

In [ ]:
shapefile.iloc[0]

In [ ]:
shapefile = shapefile[['geometry']]

5. Convert from GeoDataFrame into GeoJSON:

code snippet below based from https://gis.stackexchange.com/questions/333791/accessing-a-shapefile-with-googleearthengine-api-invalid-geojson-geometry/334400#334400


In [ ]:
geom = shapefile.iloc[0:1,:] 
jsonDict = eval(geom.to_json()) 
geojsonDict = jsonDict['features'][0] 
region = ee.FeatureCollection(ee.Feature(geojsonDict)).geometry()

In [ ]:
region.getInfo()

6. Define raster attributes.

In [ ]:
# define the satellite source: https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA
bands_for_url = [{'id':'B5'}, {'id':'B4'}, {'id':'B3'}, {'id':'B2'}, {'id':'B1'}]
bands_for_sat = ['B5', 'B4', 'B3', 'B2', 'B1']
#bands_for_sat = ['B4', 'B3', 'B2']

In [ ]:
L7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_TOA").filterMetadata('CLOUD_COVER','less_than',10).select(bands_for_sat)
L8T1 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterMetadata('CLOUD_COVER','less_than',10).select(bands_for_sat)
#cop = ee.ImageCollection("COPERNICUS/S2").filterMetadata('CLOUD_COVER','less_than',10).select(bands_for_sat)

In [ ]:
# define filters
sat_collection = L8T1
# Used 1 day as time range for testing. Please give a longer duration to generate more.
time_range = ['2021-01-01', '2021-01-02']

In [ ]:
image_area = sat_collection.filterDate(time_range[0], time_range[1]).filterBounds(region)
images = sat_collection.filterDate(time_range[0], time_range[1])
image_median = image_area.median()
clip = image_median.clip(region).toFloat()

In [ ]:
cnt = images.size().getInfo()
cnt

129

In [ ]:
images.getInfo()

### 7. Display raster, for this example using geemap and folium.  

Or, export to GDrive, 1 of 4 (Cloud, Assets, local) methods in addition to inline display.

### geemap.org
One method to display is geemap and shown below.

In [ ]:
!pip install geemap

In [ ]:
# Installs geemap package
import geemap.eefolium as geemap
Map = geemap.Map()

In [ ]:
ee_object = ee.Geometry(region)
#ee_object = ee.Geometry(geometry)
type(ee_object)

In [ ]:
Map.centerObject(region, zoom=8)
Map

In [ ]:
visParams = {'bands': ['B5'], max: 0.3};
Map.addLayer(clip, visParams, 'true-color composite');
Map.addLayerControl()
Map

# Convert ImageCollection to List

In [ ]:
image_list = sat_collection.toList(cnt)

# Saving to GDrive
# Iterate through the list, take each image and move it to G drive

In [ ]:
import time

# I have hardcoded endrange as 2, please change it to cnt
# so that the entire list will be iterated
for i in range(0, 2):
  image = ee.Image(image_list.get(i))
  #image_clip = image.clip(ee.Geometry(image.geometry())).toFloat()
  print('Name: ', name)

  task_config = {'region' : region,
                'folder' : 'satellite_images',
                'scale' : 10,
                'crs':'EPSG:4326',
                'fileFormat' : 'GeoTIFF'}

  task = ee.batch.Export.image.toDrive(image, #current image
                                         "l8_imagery_" + str(i), #name of file
                                        **task_config)
  task.start()
  print(task.status())
  while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(20)

Name:  LC08_001004_20150714
{'state': 'READY', 'description': 'l8_imagery_0', 'creation_timestamp_ms': 1630849710486, 'update_timestamp_ms': 1630849710486, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'QE6FXGVCJBMHIYVZFTAJIX2S', 'name': 'projects/earthengine-legacy/operations/QE6FXGVCJBMHIYVZFTAJIX2S'}
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX2S).
Polling for task (id: QE6FXGVCJBMHIYVZFTAJIX

# Visualize the generated images using rasterio - On visualizing, looks like the region we pass while extracting the image is incorrect. Needs to be corrected.

In [ ]:
!pip -q install rasterio

In [ ]:
%matplotlib inline

import rasterio
from matplotlib import pyplot
from rasterio.plot import show

image = rasterio.open('./drive/My Drive/Netherlands/satellite_images/l8_image_0.tif')

In [ ]:
show(image)

In [ ]:
show(image.read(2), transform=image.transform, cmap="viridis")

In [ ]:
pyplot.show()